In [33]:

import time
import traceback
import csv

from datetime import datetime, timedelta
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select

from bs4 import BeautifulSoup

from itertools import combinations

def parse_date(date_: datetime):
    '''datetime 'yyyy-mm-dd' 형식 str으로 반환'''
    return date_.strftime(r"%Y-%m-%d")

def hotel_crawl(s_date: datetime, e_date: datetime):
    '입력한 시간/날짜에 따라 크롤링 진행'
    driver.get('https://www.booking.com/searchresults.ko.html?ss=%EC%A0%9C%EC%A3%BC%EB%8F%84%2C+%EB%8C%80%ED%95%9C%EB%AF%BC%EA%B5%AD&label=Korean_South_Korea-KO-KR-131246354044-lqHqiW6tNbRkHjsFQdmmhwS637818828162%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atidsa-1227182654382%3Alp1009842%3Ali%3Adec%3Adm%3Aag131246354044%3Acmp400536625&sid=1a2cdd5c26bf10a9c3ed2baaa7785c56&aid=318615&lang=ko&sb=1&src_elem=sb&src=index&dest_id=4170&dest_type=region&group_adults=2&no_rooms=1&group_children=0&sb_travel_purpose=leisure')

    print(f'{s_date}에서 {e_date} 숙박 조회시작')
    time.sleep(10)
    
    # 날짜 선택
    try: 
        # 시작일
        date_1 = driver.find_element(By.XPATH, rf'//*[@data-date="{parse_date(s_date)}"]')
        date_1.click()
        time.sleep(0.5)

        # 종료일
        date_2 = driver.find_element(By.XPATH, rf'//*[@data-date="{parse_date(e_date)}"]')
        date_2.click()
        time.sleep(0.5)

        # 적용하기 클릭
        done = driver.find_element(By.XPATH, rf'//*[@type="submit"]')
        done.click()
        time.sleep(0.5)
        
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        
        prod_list = soup.select('#property-card div')
    except Exception as error:
        print(f'에러: {type(error).__name__}, 메시지: {error}')
        #hotel_csv(prod_list, s_date, e_date)
def get_date_combinations(start_date, days=30):
    'start_date부터 days일 후까지 가능한 시작/종료일 조합'
    date_list = []
    for i in range(days):
        current_date = start_date + timedelta(days=i)
        date_list.append(current_date)
    
    combinations_list = list(combinations(date_list, 2))
    return combinations_list



In [34]:
columns = ['hotel_name','region','ratings', 'price', 'start_date', 'end_date']

In [35]:
def hotel_csv(prod_list, s_date, e_date):
    'csv로 저장'
    file_name = 'hotels.csv'
    file_exists = False

    # 이미 파일 있으면 추가하기 위한 작업
    try:
        with open(file_name, 'r', encoding='utf-8') as csvfile:
            reader = csv.reader(csvfile)
            file_exists = any(row for row in reader) # 파일에 데이터가 있는지 확인합니다.
    except FileNotFoundError:
        pass


    data_list = []
    rental_dates = [s_date, e_date]
    for section in prod_list:
        hotel_name = section.select_one('[data-testid="title"]').text.strip()
        region = section.select_one('[data-testid="address"]').text.strip()
        ratings = float(section.select_one('[aria-label="7.8"]').next_sibling.strip())
        price = int(section.select_one('[data-testid="title"]').text.strip().replace(',',''))
        data_list.append([car_name,
                              region,
                              ratings,
                              price]
                              + rental_dates)

    with open('hotel.csv', 'a+', encoding='utf-8', newline='') as file:
        csv_writer = csv.writer(file)
        if not file_exists:
            csv_writer.writerow(columns)
        csv_writer.writerows(data_list)

if __name__ == '__main__':
    driver = webdriver.Chrome()
    driver.maximize_window()

    s_date = datetime.today() + timedelta(days=1)
    date_combos = get_date_combinations(s_date)



    for combo in date_combos:
        hotel_crawl(combo[0], combo[1])

    driver.quit()

    

2023-07-22 11:14:36.796225에서 2023-07-23 11:14:36.796225 숙박 조회시작
2023-07-22 11:14:36.796225에서 2023-07-24 11:14:36.796225 숙박 조회시작
에러: NoSuchElementException, 메시지: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@data-date="2023-07-22"]"}
  (Session info: chrome=115.0.5790.99); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00B2A813+48355]
	(No symbol) [0x00ABC4B1]
	(No symbol) [0x009C5358]
	(No symbol) [0x009F09A5]
	(No symbol) [0x009F0B3B]
	(No symbol) [0x00A1E232]
	(No symbol) [0x00A0A784]
	(No symbol) [0x00A1C922]
	(No symbol) [0x00A0A536]
	(No symbol) [0x009E82DC]
	(No symbol) [0x009E93DD]
	GetHandleVerifier [0x00D8AABD+2539405]
	GetHandleVerifier [0x00DCA78F+2800735]
	GetHandleVerifier [0x00DC456C+2775612]
	GetHandleVerifier [0x00BB51E0+616112]
	(No symbol) [0x00AC5F8C]
	(No symbol) [0x00AC2328]
	(No symb

KeyboardInterrupt: 